Para criar um modelo de Machine Learning para classificar se a água é potável ou não com base no dataset fornecido, e fazer o tracking e registro do modelo utilizando MLflow, serão seguidos os seguintes passos:

1) Análise Exploratória dos Dados: Primeiro o dataset será explorado para entender suas características, como número de amostras, distribuição dos dados, valores ausentes, etc.
2) Pré-processamento dos Dados: Esta etapa envolve lidar com valores ausentes, normalizar ou padronizar os dados, e possivelmente selecionar ou extrair características relevantes.
3) Construção do Modelo: Um modelo de ML apropriado será escolhido para classificação binária.
4) Avaliação do Modelo: Será avaliada as métricas como acurácia, precisão, recall e F1-score.
5) Tracking e Registro com MLflow: O MLflow será usado para fazer o tracking dos experimentos, incluindo parâmetros do modelo, métricas de desempenho e o próprio modelo treinado.
6) Boas Práticas de Programação: Durante todo o processo, será garantido que o código siga as boas práticas, como uso de Design Patterns, documentação adequada e evitando importações desnecessárias.

## Análise exploratória dos dados

In [22]:
import pandas as pd

# Carregando o dataset
file_path = "dados/water_potability.csv"
data = pd.read_csv(file_path)

# Exibindo as primeiras linhas do dataset para análise inicial
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [23]:
# Checando dados faltantes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [24]:
# Checando descritivo estatístico
data.drop(columns=["Potability"]).describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000


In [25]:
# Contagem da variável dependente
data["Potability"].value_counts()

0    1998
1    1278
Name: Potability, dtype: int64

A análise inicial do dataset revela as seguintes características:

**Colunas:** O dataset possui colunas para pH, Hardness (Dureza), Solids (Sólidos), Chloramines (Cloraminas), Sulfate (Sulfato), Conductivity (Condutividade), Organic_carbon (Carbono Orgânico), Trihalomethanes (Trihalometanos) e Turbidity (Turbidez), além da coluna alvo, Potability, que indica se a água é potável (1) ou não (0).

**Dados Faltantes:** Há valores ausentes em algumas colunas, como pH e Sulfate.
O próximo passo será tratar os dados faltantes e normalizar os dados. Para isso, podemos imputar os valores faltantes com a média da coluna e depois normalizar todos os recursos para ter uma média de 0 e um desvio padrão de 1, o que é uma prática comum em modelos de Machine Learning.

## Pré-processamento dos dados

O próximo passo será tratar os dados faltantes e normalizar os dados. Para isso, os valores faltantes serão imputados com a média da coluna e depois serão normalizados todos os recursos para ter uma média de 0 e um desvio padrão de 1, o que é uma prática comum em modelos de Machine Learning.

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Imputação de valores faltantes com a média da coluna
imputer = SimpleImputer(strategy="mean")
data_imputed = imputer.fit_transform(data.drop("Potability", axis=1))

# Normalização dos dados
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_imputed)

# Preparando os dados para treinamento e teste
X = data_normalized
y = data["Potability"]

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((2293, 9), (983, 9), (2293,), (983,))

Os dados foram pré-processados com sucesso:

**Imputação:** Valores faltantes foram imputados usando a média da respectiva coluna.

**Normalização:** Os dados foram normalizados para ter uma média de 0 e um desvio padrão de 1.
Divisão do Dataset: O conjunto de dados foi dividido em conjuntos de treinamento (2293 amostras) e teste (983 amostras).

## Construção do modelo, avaliação, tracking e registro com MLflow

Agora o modelo de Machine Learning será implementado. Será utilizado um modelo simples, a Regressão Logística, que é eficaz para problemas de classificação binária. Posteriormente, será feita a avaliação do modelo e utilizaremos o MLflow para fazer o tracking do experimento. Vamos prosseguir com a construção e avaliação do modelo. 

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn

# Configurando o experimento no MLflow
mlflow.set_experiment("Water Potability Classification")

# Construindo e treinando o modelo de Regressão Logística
with mlflow.start_run(run_name="logistic-regression-model"):
    model = LogisticRegression(random_state=123)
    model.fit(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Calculando métricas de avaliação
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Registrando parâmetros, métricas e modelo no MLflow
    mlflow.log_param("model", "Logistic Regression")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    mlflow.sklearn.log_model(model, "model")
